This file creates pandas dataframes from rosbag files.

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os

import rospy
import rosbag
from bagpy import bagreader

### Create topic list using bagpy

In [2]:
# This is not needed for the actual evaluation. For fast topic overview only.
# If problems with bagpy exist, just comment or leave out.
bag_nr = 2
sloppy_joystick_bag = bagreader('../../rosbags/participent_' + str(bag_nr) + '/sloppy_joystick.bag')
pd.set_option('display.max_rows', 500)
display(sloppy_joystick_bag.topic_table)

[INFO]  Data folder ../../rosbags/participent_2/sloppy_joystick already exists. Not creating.


,Topics,Types,Message Count,Frequency
0,/amcl/parameter_descriptions,dynamic_reconfigure/ConfigDescription,1,NaN
1,/amcl/parameter_updates,dynamic_reconfigure/Config,1,NaN
2,/amcl_pose,geometry_msgs/PoseWithCovarianceStamped,94,1.320241
3,/battery_state,sensor_msgs/BatteryState,2231,22.011220
4,/cmd_vel,geometry_msgs/Twist,8574,98.098606
5,/cmd_vel_rc100,geometry_msgs/Twist,2231,22.010180
6,/col_cmd,geometry_msgs/Twist,429,4.982128
7,/diagnostics,diagnostic_msgs/DiagnosticArray,254,3.217093
8,/firmware_version,turtlebot3_msgs/VersionInfo,87,0.993989
9,/homing_cmd,geometry_msgs/Twist,8413,96.694193


### define aux. functions

In [3]:
# https://automaticaddison.com/how-to-convert-a-quaternion-into-euler-angles-in-python/
import math
def euler_from_quaternion(x, y, z, w):
    """
    Convert a quaternion into euler angles (roll, pitch, yaw)
    roll is rotation around x in radians (counterclockwise)
    pitch is rotation around y in radians (counterclockwise)
    yaw is rotation around z in radians (counterclockwise)
    """
    t0 = +2.0 * (w * x + y * z)
    t1 = +1.0 - 2.0 * (x * x + y * y)
    roll_x = math.atan2(t0, t1)  
    t2 = +2.0 * (w * y - z * x)
    t2 = +1.0 if t2 > +1.0 else t2
    t2 = -1.0 if t2 < -1.0 else t2
    pitch_y = math.asin(t2)
    t3 = +2.0 * (w * z + x * y)
    t4 = +1.0 - 2.0 * (y * y + z * z)
    yaw_z = math.atan2(t3, t4)
    return roll_x, pitch_y, yaw_z # in radians


def create_df_from_twist(bag, twist_topic):
    twist_df = pd.DataFrame(columns=['time', 'v', 'omega'])
    for topic, msg, t in bag.read_messages(topics=[twist_topic]):
        time = rospy.Time.to_sec(t) # convert to float
        v = msg.linear.x
        omega = msg.angular.z
        twist_df = twist_df.append({'time': time, 'v': v, 'omega': omega}, ignore_index=True)
    return twist_df

def create_df_from_laserscan(bag, laserscan_topic, t0=0): # t0 is a time offset that is subtracted
    scan_df = pd.DataFrame(columns=['time', 'ranges', 'angles'])
    for topic, msg, t in bag.read_messages(topics=[laserscan_topic]):
        time = (rospy.Time.to_sec(t) - t0) / 1000000 # time in nano-seconds (10^-9) - > / 10^6 -> milli-seconds
        ranges = msg.ranges
        ranges = np.array(ranges)
        angles = np.arange(msg.angle_min, msg.angle_max+msg.angle_increment, msg.angle_increment)
        angles = angles[ranges > msg.range_min]
        ranges = ranges[ranges > msg.range_min] 
        scan_df = scan_df.append({'time': time, 'ranges': ranges, 'angles': angles}, ignore_index=True)
    return scan_df

def create_df_from_odometry(bag, odometry_topic, t0=0): # t0 is a time offset that is subtracted
    odom_df = pd.DataFrame(columns=['time', 'x', 'y', 'yaw', 'v', 'omega'])
    for topic, msg, t in bag.read_messages(topics=[odometry_topic]):
        time = (rospy.Time.to_sec(t) - t0) / 1000000 # time in nano-seconds (10^-9) - > / 10^6 -> milli-seconds
        
        x = msg.pose.pose.position.x
        y = msg.pose.pose.position.y
        roll, pitch, yaw = euler_from_quaternion(msg.pose.pose.orientation.x, msg.pose.pose.orientation.y, msg.pose.pose.orientation.z, msg.pose.pose.orientation.w)
        v = msg.twist.twist.linear.x
        omega = msg.twist.twist.angular.z

        odom_df = odom_df.append({'time': time, 'x': x, 'y': y, 'yaw': yaw, 'v': v, 'omega': omega}, ignore_index=True)
    return odom_df

### specify data

In [4]:
# choose participent
participent_nr = 5 

# choose rosbags
# bag_names = ["sloppy_joystick"] # participent 1 only has this bag
bag_names = ["main_circuit", "main_circuit_second_target", "normal_joystick", "normal_joystick_second_target", "sloppy_joystick", "sloppy_joystick_second_target"]

# list twist topic names
twist_topic_list = ['/sloppy_joy_cmd', '/nav_cmd', '/joy_cmd', '/homing_cmd', '/cmd_vel_rc100', '/cmd_vel', '/col_cmd']

### create pandas dataframes (df)

In [5]:
from pathlib import Path # Python ≥ 3.5


for bag_name in bag_names:
    
    folder_path_df = "../df/participent_" + str(participent_nr) + '/' + bag_name  
    Path(folder_path_df).mkdir(parents=True, exist_ok=True)
    bag = rosbag.Bag('../../rosbags/participent_' + str(participent_nr) + '/' + bag_name + '.bag')
    
    # create a pandas dataframe (df) for every twist topic
    for twist_topic_name in twist_topic_list:
        twist_df = create_df_from_twist(bag, twist_topic_name)
        file_path = folder_path_df + '/' + twist_topic_name + ".csv"
        twist_df.to_csv(file_path, index = False, header=True)
        
    # create a pandas dataframe (df) for laserscan topic
    laserscan_df = create_df_from_laserscan(bag, "/scan")
    laserscan_df.to_csv(folder_path_df + "/scan.csv", index = False, header=True)
    
    # create a pandas dataframe (df) for odom topic
    odom_df = create_df_from_odometry(bag, "/odom")
    odom_df.to_csv(folder_path_df + "/odom.csv", index = False, header=True)

### create html files

In [6]:
# create a HTML file for easy access (without jupyter)
os.system('jupyter nbconvert --to html preprocessing.ipynb') 
os.system('mv preprocessing.html ../html') # move to html folder

0